# AES Key Recovery: full key

## Description

In this attack, we aim to recover the first bit of the key using a binary classifier. We train a neural network on plaintext-ciphertext pairs as samples and the first bit of the key as labels.

## Imports

In [35]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from dataset.datasets import AESDatasetCiphertextPlaintextPairKey
from pipeline import *

## Importing the dataset

In [17]:
data = AESDatasetCiphertextPlaintextPairKey(128, 'large')

train_labels, train_samples, test_labels, test_samples = data.get_data()

Error: dataset/assets/moby-dick.txt
Error: dataset/assets/shakespeare.txt


In [18]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (1445705, 128)
===== Label Shape: (128,)
===== Training Samples Shape: (1445705, 256)
===== Sample Shape: (256,)
===== Testing Labels Shape: (619588, 128)
===== Testing Samples Shape: (619588, 256)


### Preprocessing

We add 1 preprocessing step:
As our goal is only to recover the first bit of the key, we only keep the first bit of the labels to train our network.


## Creating the model

In [19]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

Add customization here.

In [29]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = len(train_labels[0])

# units per hidden layer
units = 1024

# 0.1 to 0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=10000,
    decay_rate=0.01)

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
learning_rate = 0.001
optimizer = Adam(learning_rate=lr_schedule)
metrics = ['binary_accuracy']
epochs = 50
batch_size = 1000

### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [30]:
# Type of model
neural_network = Sequential()

# Input layer
neural_network.add(Input(shape=input_shape))

# Hidden layers
#neural_network.add(BatchNormalization())
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))

# Output layer
neural_network.add(Dense(units=dim, activation='sigmoid'))

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_mse, metrics=metrics)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 1024)              263168    
                                                                 
 dense_21 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_22 (Dense)            (None, 128)               131200    
                                                                 
Total params: 1,443,968
Trainable params: 1,443,968
Non-trainable params: 0
_________________________________________________________________


### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [31]:
history = train_model(neural_network, train_samples, train_labels, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/50
1446/1446 [==============================] - 54s 37ms/step - loss: 0.2500 - binary_accuracy: 0.5000
Epoch 2/50
1446/1446 [==============================] - 52s 36ms/step - loss: 0.2500 - binary_accuracy: 0.5000
Epoch 3/50
1446/1446 [==============================] - 52s 36ms/step - loss: 0.2500 - binary_accuracy: 0.4999
Epoch 4/50
1446/1446 [==============================] - 52s 36ms/step - loss: 0.2500 - binary_accuracy: 0.5000
Epoch 5/50
1446/1446 [==============================] - 52s 36ms/step - loss: 0.2500 - binary_accuracy: 0.5000
Epoch 6/50
1446/1446 [==============================] - 52s 36ms/step - loss: 0.2500 - binary_accuracy: 0.5000
Epoch 7/50
1446/1446 [==============================] - 52s 36ms/step - loss: 0.2500 - binary_accuracy: 0.5000
Epoch 8/50
1446/1446 [==============================] - 52s 36ms/step - loss: 0.2500 - binary_accuracy: 0.5000
Epoch 9/50
1446/1446 [==============================] - 55s 38ms/step - loss: 0.2500 - binary_accuracy: 0.5001
E

### Testing
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [32]:
results = test_model(neural_network, test_samples, test_labels, batch_size, ascii_correction=False)

620/620 [==============================] - 8s 12ms/step


In [33]:
print("=====================================")
print("Correct predictions: " + str(results["correct_predictions"]))
print("Accuracy: " + str(results["accuracy"]))
print("=====================================")
print("Correct bytes: " + str(results["correct_bytes"]))
print("Byte Accuracy: " + str(results["byte_accuracy"]))
print("=====================================")

Correct predictions: 0
Accuracy: 0.0
Correct bytes: 0
Byte Accuracy: 0.0
